In [11]:
import numpy as np
import sys
from ipycanvas import Canvas
from tqdm import tqdm
sys.path.append('../src')
from util import draw, grab_plot, plot, RunTime, VideoWriter

In [83]:
CANVAS_SIZE = 500

def update():
    global farm, canvas, x, v, a

    x, v = farm(x, v, a)
    draw(x, canvas, CANVAS_SIZE, ant_size=10)

def toggle(*_):
    global runtime 

    if runtime.running:
        runtime.stop()
    else:
        runtime.start()

def simulate(function, interval=.02):
    global runtime, canvas, x

    display(canvas)
    draw(x, canvas, CANVAS_SIZE, ant_size=10)

    try:
        runtime.stop()
    except:
        pass

    runtime = RunTime(function, interval)

def record(n_steps=500):
    global farm, x, v

    with VideoWriter(size=CANVAS_SIZE) as vid:
        for _ in tqdm(range(n_steps), desc='Ant Farm'):
            plot(x)
            vid.write(grab_plot())
            x, v = farm(x, v)

In [84]:
class SandboxFarm:
    def __init__(self, dt=.1, figsize=500):
        self.dt = dt
        self.figsize = figsize

    def __call__(self, x, v, a):
        v = v + a*self.dt

        for i in range(x.shape[0]):
            if x[i, 1] >= self.figsize - 5.:
                v[i, 1] = -.9*v[i, 1]

        x = (x + v*self.dt + .5*a*self.dt**2)%self.figsize
        
        return x, v

In [86]:
N_ANTS = 1

farm = SandboxFarm(figsize=CANVAS_SIZE)
x = .5*CANVAS_SIZE*np.ones((N_ANTS, 2))
v = np.zeros_like(x)
a = np.array([[0., 9.8],]*N_ANTS)
canvas = Canvas(width=CANVAS_SIZE, height=CANVAS_SIZE)
canvas.on_mouse_down(toggle)

simulate(update)

Canvas(width=500)